#### Import libraries

In [2]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [3]:
df = pd.read_csv('realtor-data.csv')

#### Raw data

In [4]:
df.head()

,status,bed,bath,acre_lot,city,state,zip_code,house_size,prev_sold_date,price
0,for_sale,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.0,NaN,105000.0
1,for_sale,4.0,2.0,0.08,Adjuntas,Puerto Rico,601.0,1527.0,NaN,80000.0
2,for_sale,2.0,1.0,0.15,Juana Diaz,Puerto Rico,795.0,748.0,NaN,67000.0
3,for_sale,4.0,2.0,0.10,Ponce,Puerto Rico,731.0,1800.0,NaN,145000.0
4,for_sale,6.0,2.0,0.05,Mayaguez,Puerto Rico,680.0,NaN,NaN,65000.0


In [5]:
df.shape #unclean data size

(904966, 10)

In [6]:
df.isnull().sum()

status                 0
bed               129840
bath              113884
acre_lot          266642
city                  72
state                  0
zip_code             204
house_size        292886
prev_sold_date    459101
price                 71
dtype: int64

### Data preprocessing

In [7]:
df1 = df.copy()

In [8]:
df1 = df1.drop(['prev_sold_date'],axis=1) #dropping unwanted columns - prev_sold_date

In [9]:
df1.isnull().sum()

status             0
bed           129840
bath          113884
acre_lot      266642
city              72
state              0
zip_code         204
house_size    292886
price             71
dtype: int64

In [10]:
df1['bed'].fillna(np.random.randint(1,4),inplace=True) #replacing NaN values with random(1-4) beds

In [11]:
df1 = df1.loc[df1['bed']<=4]

In [12]:
df1['bed'].unique()

array([3., 4., 2., 1.])

In [13]:
df1['bath'].fillna(np.random.randint(1,3),inplace = True)

In [14]:
df1.isnull().sum()

status             0
bed                0
bath               0
acre_lot      255689
city              62
state              0
zip_code         201
house_size    257654
price             62
dtype: int64

In [15]:
df1.head()

,status,bed,bath,acre_lot,city,state,zip_code,house_size,price
0,for_sale,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.0,105000.0
1,for_sale,4.0,2.0,0.08,Adjuntas,Puerto Rico,601.0,1527.0,80000.0
2,for_sale,2.0,1.0,0.15,Juana Diaz,Puerto Rico,795.0,748.0,67000.0
3,for_sale,4.0,2.0,0.10,Ponce,Puerto Rico,731.0,1800.0,145000.0
5,for_sale,4.0,3.0,0.46,San Sebastian,Puerto Rico,612.0,2520.0,179000.0


In [16]:
df2 = df1.copy()

In [17]:
df2 = df2.drop(df2[df2['price'].isnull()].index) #dropping Nan values in price

In [18]:
df2 = df2.drop(df2[df2['city'].isnull()].index) #dropping Nan values in city

In [19]:
df2 = df2.drop(df2[df2['zip_code'].isnull()].index) #dropping Nan values in zipcode

In [20]:
df2.isnull().sum()

status             0
bed                0
bath               0
acre_lot      255591
city               0
state              0
zip_code           0
house_size    257564
price              0
dtype: int64

In [21]:
df2

,status,bed,bath,acre_lot,city,state,zip_code,house_size,price
0,for_sale,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.0,105000.0
1,for_sale,4.0,2.0,0.08,Adjuntas,Puerto Rico,601.0,1527.0,80000.0
2,for_sale,2.0,1.0,0.15,Juana Diaz,Puerto Rico,795.0,748.0,67000.0
3,for_sale,4.0,2.0,0.10,Ponce,Puerto Rico,731.0,1800.0,145000.0
5,for_sale,4.0,3.0,0.46,San Sebastian,Puerto Rico,612.0,2520.0,179000.0
...,...,...,...,...,...,...,...,...,...
904961,for_sale,2.0,2.0,NaN,New York City,New York,10463.0,NaN,359000.0
904962,for_sale,2.0,2.0,NaN,Bronx,New York,10463.0,1200.0,299000.0
904963,for_sale,1.0,1.0,NaN,Bronx,New York,10463.0,NaN,120000.0
904964,for_sale,3.0,2.0,NaN,Bronx,New York,10463.0,NaN,449000.0


In [23]:
housing_data=df2 #imputing nan values of house_size with linear regression predicted values

#### Building a Linear regression model on missing 'house_size' values based on number of beds, number of baths, city, state since it can't be randomly populated or deleted.

In [24]:
housing_data["city_num"] = housing_data["city"].astype("category").cat.codes
housing_data["state_num"] = housing_data["state"].astype("category").cat.codes

features = ["bed", "bath", "city_num", "state_num","price"]

housing_data_no_nan = housing_data[~np.isnan(housing_data["house_size"])]

model = LinearRegression()
model.fit(housing_data_no_nan[features], housing_data_no_nan["house_size"])

house_size_imputed = housing_data["house_size"].copy()
house_size_imputed[np.isnan(house_size_imputed)] = model.predict(housing_data[features][np.isnan(house_size_imputed)])

housing_data["house_size"] = house_size_imputed

In [25]:
housing_data.isnull().sum()

status             0
bed                0
bath               0
acre_lot      255591
city               0
state              0
zip_code           0
house_size         0
price              0
city_num           0
state_num          0
dtype: int64

In [26]:
housing_data.head(60)

,status,bed,bath,acre_lot,city,state,zip_code,house_size,price,city_num,state_num
0,for_sale,3.0,2.0,0.12,Adjuntas,Puerto Rico,601.0,920.000000,105000.0,14,9
1,for_sale,4.0,2.0,0.08,Adjuntas,Puerto Rico,601.0,1527.000000,80000.0,14,9
2,for_sale,2.0,1.0,0.15,Juana Diaz,Puerto Rico,795.0,748.000000,67000.0,1042,9
3,for_sale,4.0,2.0,0.10,Ponce,Puerto Rico,731.0,1800.000000,145000.0,1705,9
5,for_sale,4.0,3.0,0.46,San Sebastian,Puerto Rico,612.0,2520.000000,179000.0,1879,9
6,for_sale,3.0,1.0,0.20,Ciales,Puerto Rico,639.0,2040.000000,50000.0,407,9
7,for_sale,3.0,2.0,0.08,Ponce,Puerto Rico,731.0,1050.000000,71600.0,1705,9
8,for_sale,2.0,1.0,0.09,Ponce,Puerto Rico,730.0,1092.000000,100000.0,1705,9
10,for_sale,3.0,2.0,13.39,Isabela,Puerto Rico,662.0,1106.000000,89000.0,1011,9
11,for_sale,3.0,2.0,0.08,Juana Diaz,Puerto Rico,795.0,1045.000000,150000.0,1042,9


In [27]:
df3 = housing_data

In [28]:
df3["acre_lot"] = df3["acre_lot"].fillna(df3["acre_lot"].median()) #population NaN acre_lot values with median of the column

In [29]:
df3.isnull().sum()

status        0
bed           0
bath          0
acre_lot      0
city          0
state         0
zip_code      0
house_size    0
price         0
city_num      0
state_num     0
dtype: int64

In [30]:
df3.dtypes

status         object
bed           float64
bath          float64
acre_lot      float64
city           object
state          object
zip_code      float64
house_size    float64
price         float64
city_num        int16
state_num        int8
dtype: object

In [31]:
df3['bed'] = df3['bed'].astype('int64') # changing the data type of bed,bath,price,zip_code, house_size from float to int
df3['bath'] = df3['bath'].astype('int64')
df3['price'] = df3['price'].astype('int64')
df3['zip_code'] = df3['zip_code'].astype('int64')
df3['house_size'] = df3['house_size'].astype('int64')

In [32]:
df3 = df3.drop(['city_num','state_num','status'],axis=1) #removing unwanted columns

In [33]:
df3

,bed,bath,acre_lot,city,state,zip_code,house_size,price
0,3,2,0.12,Adjuntas,Puerto Rico,601,920,105000
1,4,2,0.08,Adjuntas,Puerto Rico,601,1527,80000
2,2,1,0.15,Juana Diaz,Puerto Rico,795,748,67000
3,4,2,0.10,Ponce,Puerto Rico,731,1800,145000
5,4,3,0.46,San Sebastian,Puerto Rico,612,2520,179000
...,...,...,...,...,...,...,...,...
904961,2,2,0.33,New York City,New York,10463,1461,359000
904962,2,2,0.33,Bronx,New York,10463,1200,299000
904963,1,1,0.33,Bronx,New York,10463,673,120000
904964,3,2,0.33,Bronx,New York,10463,1672,449000


In [34]:
housing_data.head(60)

,status,bed,bath,acre_lot,city,state,zip_code,house_size,price,city_num,state_num
0,for_sale,3,2,0.12,Adjuntas,Puerto Rico,601,920,105000,14,9
1,for_sale,4,2,0.08,Adjuntas,Puerto Rico,601,1527,80000,14,9
2,for_sale,2,1,0.15,Juana Diaz,Puerto Rico,795,748,67000,1042,9
3,for_sale,4,2,0.10,Ponce,Puerto Rico,731,1800,145000,1705,9
5,for_sale,4,3,0.46,San Sebastian,Puerto Rico,612,2520,179000,1879,9
6,for_sale,3,1,0.20,Ciales,Puerto Rico,639,2040,50000,407,9
7,for_sale,3,2,0.08,Ponce,Puerto Rico,731,1050,71600,1705,9
8,for_sale,2,1,0.09,Ponce,Puerto Rico,730,1092,100000,1705,9
10,for_sale,3,2,13.39,Isabela,Puerto Rico,662,1106,89000,1011,9
11,for_sale,3,2,0.08,Juana Diaz,Puerto Rico,795,1045,150000,1042,9


In [35]:
df3['bath'].unique()

array([ 2,  1,  3,  4,  6,  7,  5, 10,  8,  9, 20, 29, 18, 13, 15, 11, 16,
       12, 22])

In [36]:
df3.isnull().sum()

bed           0
bath          0
acre_lot      0
city          0
state         0
zip_code      0
house_size    0
price         0
dtype: int64

In [37]:
df3 = df3.loc[df3['bath']<=4] #removing erraneous bath values

In [38]:
df3

,bed,bath,acre_lot,city,state,zip_code,house_size,price
0,3,2,0.12,Adjuntas,Puerto Rico,601,920,105000
1,4,2,0.08,Adjuntas,Puerto Rico,601,1527,80000
2,2,1,0.15,Juana Diaz,Puerto Rico,795,748,67000
3,4,2,0.10,Ponce,Puerto Rico,731,1800,145000
5,4,3,0.46,San Sebastian,Puerto Rico,612,2520,179000
...,...,...,...,...,...,...,...,...
904961,2,2,0.33,New York City,New York,10463,1461,359000
904962,2,2,0.33,Bronx,New York,10463,1200,299000
904963,1,1,0.33,Bronx,New York,10463,673,120000
904964,3,2,0.33,Bronx,New York,10463,1672,449000


#### Download clean data

In [39]:
df3.to_csv('housing_data.csv') 

#### Finding average house price per zipcode

In [289]:
df4 = df3.copy() 

In [290]:
df4.head()

,bed,bath,acre_lot,city,state,zip_code,house_size,price
0,3,2,0.12,Adjuntas,Puerto Rico,601,920,105000
1,4,2,0.08,Adjuntas,Puerto Rico,601,1527,80000
2,2,1,0.15,Juana Diaz,Puerto Rico,795,748,67000
3,4,2,0.10,Ponce,Puerto Rico,731,1800,145000
5,4,3,0.46,San Sebastian,Puerto Rico,612,2520,179000


In [291]:
df4=df4.drop(['bed','bath','acre_lot','city','state','house_size'],axis=1)

In [292]:
df4.head()

,zip_code,price
0,601,105000
1,601,80000
2,795,67000
3,731,145000
5,612,179000


In [293]:
df4 = df4.groupby('zip_code').mean() #Averaging the house price for each zipcode

In [294]:
df4.reset_index(inplace=True)

In [295]:
df4.head()

,zip_code,price
0,601,92878.787879
1,602,190157.733990
2,603,301950.596685
3,604,79900.000000
4,605,171200.000000


In [296]:
df4.dtypes

zip_code      int64
price       float64
dtype: object

In [297]:
df4['price'] = df4['price'].astype('int64')

#### Average house price per Zipcode

In [298]:
df4.head()

,zip_code,price
0,601,92878
1,602,190157
2,603,301950
3,604,79900
4,605,171200


In [300]:
def check_condition(row):
    zip_code = row['zip_code']
    price = row['price']
    
    current_index = df4.index[df4['zip_code'] == zip_code][0] # Find the indices of the neighboring rows
    prev_index = current_index - 1
    next_index = current_index + 1
    
    if prev_index >= 0 and next_index < len(df4):             # Check if the current price meets the condition
        prev_price = df4.loc[prev_index, 'price']
        next_price = df4.loc[next_index, 'price']
        return 3 * price < prev_price and 3 * price < next_price
    else:
        return False

result = df4[df4.apply(check_condition, axis=1)]

selected_zip_codes = result['zip_code'].tolist()              # Get the zip codes that meet the condition
print("Zipcodes with high disparities in average house prices that are lower than the neighbouring zipcodes-->",selected_zip_codes)


Zipcodes with high disparities in average house prices that are lower than the neighbouring zipcodes--> [656, 661, 719, 777, 910, 970, 1011, 1235, 1243, 2553, 2571, 2714, 3240, 3260, 3273, 3785, 3864, 4020, 4022, 4056, 4237, 4280, 4343, 4419, 4479, 4492, 4539, 4544, 4553, 4680, 4763, 4777, 4923, 5070, 5252, 5494, 5650, 5675, 5746, 5827, 5831, 5858, 5862, 6336, 6350, 6373, 7710, 8007, 8073, 8252, 10924, 11431, 11953, 11961, 12033, 12165, 12565, 12746, 12864, 12988, 18040, 18342, 19374, 29835, 37760]
